In [19]:
!git clone https://github.com/yossarian-z257/medical_image_as_graph.git

Cloning into 'medical_image_as_graph'...
remote: Enumerating objects: 468, done.
remote: Counting objects: 100% (314/314), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 468 (delta 36), reused 302 (delta 26), pack-reused 154
Receiving objects: 100% (468/468), 65.75 MiB | 17.14 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [20]:
%cd medical_image_as_graph
!mkdir saved_data_loader
# !rm -rf medical_image_as_graph

/content/medical_image_as_graph/medical_image_as_graph/medical_image_as_graph


In [ ]:

!pip install imageio==2.22.4
!pip install llvmlite==0.39.1
# !pip install matplotlib==3.6.2
!pip install matplotlib==3.1.3
!pip install networkx==2.8.8
!pip install numba==0.56.4
!pip install numpy
!pip install opencv-python==4.6.0.66
!pip install psutil==5.9.4
!pip install pytz==2022.6
!pip install scikit-image==0.19.3
!pip install scipy 
!pip install timm==0.6.12
!pip install torch==1.13.0
!pip install torchinfo==1.7.1
!pip install torchvision==0.14.0

import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
!pip install gdown
!pip install ipywidgets

## IMPORTS


In [30]:
from torch_geometric.data import Data
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
from torch_geometric.loader import DataLoader
import os
from sklearn.model_selection import KFold
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.nn import Sequential,GraphConv,GCNConv,TopKPooling, GatedGraphConv, JumpingKnowledge
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import numpy as np
from torch_geometric.nn import MLP, DynamicEdgeConv, global_max_pool

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import pickle
import torch_geometric.transforms as T
# from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv
import random
from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import global_mean_pool
import torch.nn as nn
from itertools import zip_longest
import ipywidgets as widgets
from IPython.display import display

## Experiment selction

In [31]:
gnn_models = ['GCN', 'GAT', 'GIN']
super_pixels = ['5', '10', '50','100','150','300']
cnn_models = ['resnet18', 'efficientnet-b0', 'densenet121']

# defaults, before selection
gnn = "GCN"
sp = "5"
feature = "resnet18"


header1 = widgets.HTML(value="<b>GNN models</b>")
header2 = widgets.HTML(value="<b>Number of Superpixels</b>")
header3 = widgets.HTML(value="<b>CNN features</b>")


dropdown1 = widgets.Dropdown(options=gnn_models)
dropdown2 = widgets.Dropdown(options=super_pixels)
dropdown3 = widgets.Dropdown(options=cnn_models)

In [34]:
def handle_dropdown1_change(change):
    global gnn
    gnn = change['new']
    
def handle_dropdown2_change(change):
    global sp
    sp = change['new']
    
def handle_dropdown3_change(change):
    global feature
    feature = change['new']
    
dropdown1.observe(handle_dropdown1_change, names='value')
dropdown2.observe(handle_dropdown2_change, names='value')
dropdown3.observe(handle_dropdown3_change, names='value')



display(widgets.VBox([header1, dropdown1]), 
        widgets.VBox([header2, dropdown2]),
        widgets.VBox([header3, dropdown3]))

In [37]:
print(gnn,sp,feature)

GAT 10 resnet18


In [39]:
from config import saved_data_loader
experiment_text = f"{sp}_{feature}"
for fl in ['train','test','val']:
    file_name = f"{fl}_dataloader_{experiment_text}"
    file_url =  saved_data_loader[file_name]
    file_name = file_name + ".pkl"
    os.system(f"curl -o {file_name} -L '{file_url}'")
os.system(f"mv *.pkl saved_data_loader/")

0

### To replicate the results in paper, use saved model, train set to false, to train from scratch set train = True/yes/1

In [40]:
train = "no"

!python main.py  --cnn_model_name $feature --gnn_model $gnn  --use_saved_state yes   --superpixel_number $sp  --train $train

starting main.py for saved state = True
Model name:  resnet18
loading data
Using saved state
Testing
outputs/GAT_10_resnet18_best_model.pt
number of paramteres for this model 35170
1 label sensitivity 0.9871794871794872
Test Loss: 0.2339
Test Acc: 0.9006 %
testing complete
reach here
Done
